# Drawing Sequence Model

## Import modules

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, GRU, Embedding, Input, concatenate
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score

## Preprocess

In [18]:
seq_data = pd.read_csv('data/draw-sequence.csv')

from data import preprocess
data = preprocess.preprocess('data/raw/CD_PD.mat')
label = data['diagnosis'].astype('int32')

# Fill NaN with -1
seq_data = seq_data.fillna(-1)
seq_data = seq_data.values[:,1:]

# Append non-time series feature
features = ['TMSE', 'age', 'gender']
n_features = len(features)
for feature in features:
    seq_data = np.c_[seq_data, data[feature]]

seq_len = 18

def split_data():
    X_train, X_test, y_train, y_test = train_test_split(seq_data, label, test_size=0.2)

    # Reshape to 3D
    X_train_ts =  X_train[:,:seq_len].reshape((-1, seq_len, 1))
    X_test_ts = X_test[:,:seq_len].reshape((-1, seq_len, 1))
    X_train_nts = X_train[:,seq_len:].reshape((-1, n_features))
    X_test_nts = X_test[:,seq_len:].reshape((-1, n_features))

    return X_train_ts, X_test_ts, X_train_nts, X_test_nts, y_train, y_test

X_train_ts, X_test_ts, X_train_nts, X_test_nts, y_train, y_test = split_data()

## Define Model

In [23]:
def get_model(seq_len, n_features):
    inputs_seq = Input(shape=(seq_len, 1))
    x = GRU(100, return_sequences=True)(inputs_seq)
    x = Dropout(0.2)(x)
    x = GRU(100, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(1)(x)
    x = Model(inputs=inputs_seq, outputs=x)

    inputs = Input(shape=(n_features, ))
    y = Dense(100, activation='relu')(inputs)
    y = Dropout(0.2)(y)
    y = Dense(100, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(100, activation='relu')(y)
    y = Dense(1)(y)
    y = Model(inputs=inputs, outputs=y)

    combined = concatenate([x.output, y.output])
    z = Dense(2)(combined)
    z = Dense(1, activation='sigmoid')(z)

    model = Model(inputs=[x.input, y.input], outputs=z)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-5)
    optimizer=Adam(lr=1e-4)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    return model

get_model(seq_len, n_features).summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           (None, 18, 1)        0                                            
__________________________________________________________________________________________________
input_40 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
gru_39 (GRU)                    (None, 18, 100)      30600       input_39[0][0]                   
__________________________________________________________________________________________________
dense_135 (Dense)               (None, 100)          400         input_40[0][0]                   
__________________________________________________________________________________________________
dropout_77

In [26]:
model = get_model(seq_len, n_features)
model.fit([X_train_ts, X_train_nts], y_train, batch_size=32, 
    epochs=300, verbose=1, validation_data=([X_test_ts, X_test_nts], y_test))
model.evaluate([X_test_ts, X_test_nts], y_test, verbose=0)

 val_loss: 0.6235
Epoch 107/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6518 - val_loss: 0.6223
Epoch 108/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6479 - val_loss: 0.6216
Epoch 109/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6656 - val_loss: 0.6211
Epoch 110/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6596 - val_loss: 0.6208
Epoch 111/300
156/156 [==============================] - 0s 2ms/step - loss: 0.7163 - val_loss: 0.6207
Epoch 112/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6668 - val_loss: 0.6201
Epoch 113/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6363 - val_loss: 0.6200
Epoch 114/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6358 - val_loss: 0.6199
Epoch 115/300
156/156 [==============================] - 0s 2ms/step - loss: 0.6295 - val_loss: 0.6190
Epoch 116/300
156/156 [==============================] 

0.620327091217041

## Evaluation

In [27]:
def eval_score(model, X_test_ts, X_test_nts, y_test, THRESHOLD = 0.5, report = False):
    y_pred = model.predict([X_test_ts, X_test_nts], batch_size=64, verbose=1)
    y_pred_bool = np.where(y_pred < THRESHOLD, 0, 1).reshape(-1)
    if report:
        print(classification_report(y_test, y_pred_bool, target_names=['No PD', 'PD']))
    acc = accuracy_score(y_test, y_pred_bool)
    return acc

eval_score(model, X_test_ts, X_test_nts, y_test, report=True)

40/40 [==============================] - 2s 41ms/step
              precision    recall  f1-score   support

       No PD       0.66      0.86      0.75        22
          PD       0.73      0.44      0.55        18

   micro avg       0.68      0.68      0.68        40
   macro avg       0.69      0.65      0.65        40
weighted avg       0.69      0.68      0.66        40



0.675

In [28]:
scores = []
ITER_COUNT = 100
for i in range(ITER_COUNT):
    print('--------- %d try ---------' % (i + 1,))
    X_train_ts, X_test_ts, X_train_nts, X_test_nts, y_train, y_test = split_data()
    model = get_model(seq_len, n_features)
    model.fit([X_train_ts, X_train_nts], y_train, batch_size=32, epochs=150, verbose=0)
    acc = eval_score(model, X_test_ts, X_test_nts, y_test)
    scores.append(acc)
    print('Acc =', acc)
print('Min:', np.min(scores))
print('Max:', np.max(scores))
print('Avg:', np.mean(scores))
print('Std:', np.std(scores))

--------- 1 try ---------
40/40 [==============================] - 2s 41ms/step
Acc = 0.7
--------- 2 try ---------
40/40 [==============================] - 2s 52ms/step
Acc = 0.725
--------- 3 try ---------
40/40 [==============================] - 2s 41ms/step
Acc = 0.675
--------- 4 try ---------
40/40 [==============================] - 2s 43ms/step
Acc = 0.8
--------- 5 try ---------
40/40 [==============================] - 2s 46ms/step
Acc = 0.675
--------- 6 try ---------
40/40 [==============================] - 2s 49ms/step
Acc = 0.675
--------- 7 try ---------
40/40 [==============================] - 2s 49ms/step
Acc = 0.575
--------- 8 try ---------
40/40 [==============================] - 2s 53ms/step
Acc = 0.775
--------- 9 try ---------
40/40 [==============================] - 2s 60ms/step
Acc = 0.75
--------- 10 try ---------
40/40 [==============================] - 3s 69ms/step
Acc = 0.75
--------- 11 try ---------
40/40 [==============================] - 3s 73ms/step
Acc 